# Orientation Self-Test

Standalone orientation audit and polarity demo extracted from **Filament Removal.ipynb**.

Requires two local FITS files:
- AIA 193: `/Users/aosh/Downloads/AIA20240731_1800_0193.fits`
- HMI: `/Users/aosh/Downloads/hmi.M_720s.20240731_180000_TAI.fits`

In [ ]:
from pathlib import Path

import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from reproject import reproject_interp
from scipy import ndimage as ndi
import sunpy.map
from sunpy.coordinates import HeliographicCarrington, frames
from sunpy.map import all_coordinates_from_map
from sunpy.visualization import colormaps as cm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Library.Config import *
from Library.IO import *
from Library.Model import *
from Library.Processing import *

from Models import load_date_range

## Model & FITS paths

In [ ]:
ARCH_ID = "A2"
DATE_ID = "D2"
model = load_trained_model(ARCH_ID, DATE_ID)

aia_path = "/Users/aosh/Downloads/AIA20240731_1800_0193.fits"
hmi_path = "/Users/aosh/Downloads/hmi.M_720s.20240731_180000_TAI.fits"

## Helper functions

Notebook-local helpers copied from *Filament Removal.ipynb*.
These are **not** in the Library because they are analysis-specific.

In [ ]:
def compute_heliocentric_angles(aia_map):
    """
    Compute heliocentric angle \u03b8 (degrees) and cos(\u03b8) for every pixel.

    Returns
    -------
    theta_deg : 2-D array (flipped to match imshow orientation)
    cos_theta : 2-D array
    """
    coords = all_coordinates_from_map(aia_map)
    hpc = coords.transform_to(
        frames.Helioprojective(observer=aia_map.observer_coordinate)
    )
    rho_deg = np.sqrt(hpc.Tx.to(u.deg).value ** 2 + hpc.Ty.to(u.deg).value ** 2)
    r_sun_deg = float(aia_map.rsun_obs.to(u.deg).value)
    sin_theta = np.clip(rho_deg / r_sun_deg, 0, 1)
    theta_rad = np.arcsin(sin_theta)
    theta_deg = np.flipud(np.degrees(theta_rad))
    cos_theta = np.flipud(np.cos(theta_rad))
    return theta_deg, cos_theta


def apply_angle_correction(
    hmi, theta_deg, cos_theta, angle_correction=True, mask_over_60=False
):
    if mask_over_60:
        angle_cutoff = 60.0
    elif angle_correction:
        angle_cutoff = 80.0
    else:
        angle_cutoff = None

    angle_mask = np.ones_like(hmi, dtype=bool)
    if angle_cutoff is not None:
        angle_mask = theta_deg <= angle_cutoff

    hmi_out = hmi.copy()
    if angle_correction:
        hmi_out = np.where(angle_mask, hmi_out / cos_theta, 0.0).astype(np.float32)

    return hmi_out, angle_mask, angle_cutoff


def compute_elongation(binary_mask):
    ys, xs = np.where(binary_mask)
    if len(ys) < 2:
        return 0.0
    cy, cx = ys.mean(), xs.mean()
    dy, dx = ys - cy, xs - cx
    Ixx = np.mean(dx * dx)
    Iyy = np.mean(dy * dy)
    Ixy = np.mean(dx * dy)
    T = Ixx + Iyy
    D = Ixx * Iyy - Ixy * Ixy
    disc = max(T * T / 4 - D, 0.0)
    lam1 = T / 2 + np.sqrt(disc)
    lam2 = T / 2 - np.sqrt(disc)
    major = np.sqrt(max(lam1, 0.0))
    minor = np.sqrt(max(lam2, 0.0))
    if major == 0:
        return 0.0
    return float(1.0 - minor / major)


def extract_hole_data(labeled, n_holes, hmi_data):
    holes = []
    for hole_id in range(1, n_holes + 1):
        hole_mask = labeled == hole_id
        hmi_values = hmi_data[hole_mask]
        centroid = ndi.center_of_mass(hole_mask)
        elongation = compute_elongation(hole_mask)
        holes.append(
            {
                "label": hole_id,
                "hmi_values": hmi_values,
                "area_px": int(hole_mask.sum()),
                "centroid": centroid,
                "mean_hmi": float(hmi_values.mean()),
                "std_hmi": float(hmi_values.std()),
                "elongation": elongation,
            }
        )
    return holes


def plot_multiband(
    aia, hmi, nn_labeled, nn_holes,
    aia304=None, has_304=False,
    corr_label="B_los", mask_label="",
    aia_map=None, filaments_df=None,
):
    hmi_norm = np.clip(hmi, -1000, 1000)
    hmi_norm = (hmi_norm + 1000) / 2000
    n_images = 3 if has_304 else 2
    fig = plt.figure(figsize=(5 * n_images, 5))

    ax = fig.add_subplot(1, n_images, 1)
    ax.imshow(aia, cmap=cm.cmlist.get("sdoaia193"))
    _overlay_contours(ax, nn_labeled, nn_holes, show_labels=True)
    if filaments_df is not None and aia_map is not None:
        _overlay_filaments(ax, aia_map, filaments_df, color="lime")
    ax.set_title(f"AIA 193 \u00c5 | {len(nn_holes)} holes (U-Net)")
    ax.axis("off")

    if has_304 and aia304 is not None:
        ax = fig.add_subplot(1, n_images, 2)
        ax.imshow(aia304, cmap=cm.cmlist.get("sdoaia304"))
        _overlay_contours(ax, nn_labeled, nn_holes)
        if filaments_df is not None and aia_map is not None:
            _overlay_filaments(ax, aia_map, filaments_df, color="lime")
        ax.set_title("AIA 304 \u00c5")
        ax.axis("off")

    ax = fig.add_subplot(1, n_images, n_images)
    ax.imshow(hmi_norm, cmap=cm.cmlist["hmimag"])
    _overlay_contours(ax, nn_labeled, nn_holes)
    if filaments_df is not None and aia_map is not None:
        _overlay_filaments(ax, aia_map, filaments_df, color="lime")
    ax.set_title(f"HMI ({corr_label}){mask_label}")
    ax.axis("off")

    plt.tight_layout()
    plt.show()


def _overlay_contours(ax, labeled, holes, show_labels=False):
    for hole in holes:
        mask = (labeled == hole["label"]).astype(float)
        color = "red" if hole["mean_hmi"] >= 0 else "blue"
        ax.contour(mask, levels=[0.5], colors=color, linewidths=2)
        if show_labels:
            cy, cx = hole["centroid"]
            ax.text(
                cx, cy, str(hole["label"]),
                fontsize=12, color="white", ha="center", va="center",
                weight="bold",
                bbox=dict(boxstyle="circle", facecolor="orange", alpha=0.7),
            )


def _overlay_filaments(ax, aia_map, filaments_df, color="lime", lw=1.5, alpha=0.9):
    """Draw Kislovodsk filament segments on the given axes.

    world_to_pixel returns FITS-convention y (row 0 = south).
    The displayed data has been np.flipud'd (row 0 = north) and is
    shown with origin='upper', so we convert:  y_disp = (Ny-1) - y_fits.
    """
    if filaments_df is None or aia_map is None or len(filaments_df) == 0:
        return 0

    Ny = aia_map.data.shape[0]

    try:
        observer = aia_map.observer_coordinate
    except Exception:
        observer = "earth"
    obstime = aia_map.date

    n_plotted = 0
    for _, fil in filaments_df.iterrows():
        try:
            frame = HeliographicCarrington(observer=observer, obstime=obstime)
            c1 = SkyCoord(lon=fil.lon1 * u.deg, lat=fil.lat1 * u.deg, frame=frame)
            c2 = SkyCoord(lon=fil.lon2 * u.deg, lat=fil.lat2 * u.deg, frame=frame)
            px1 = aia_map.world_to_pixel(c1)
            px2 = aia_map.world_to_pixel(c2)
            y1 = (Ny - 1) - px1.y.value
            y2 = (Ny - 1) - px2.y.value
            ax.plot(
                [px1.x.value, px2.x.value],
                [y1, y2],
                color=color, lw=lw, alpha=alpha,
            )
            n_plotted += 1
        except Exception:
            continue

    return n_plotted

## Standalone polarity plot

In [ ]:
# --- Quick standalone polarity plot for two local FITS files ---
fake_row = pd.Series({"fits_path": aia_path, "hmi_path": hmi_path})

# 1. Load AIA + reproject HMI
_, aia = prepare_fits(fake_row.fits_path)
aia_map = sunpy.map.Map(fake_row.fits_path)
hmi_map = sunpy.map.Map(fake_row.hmi_path)
hmi_reprojected, _ = reproject_interp(hmi_map, aia_map.wcs, shape_out=aia.shape)
hmi = np.flipud(hmi_reprojected.astype(np.float32))
hmi = np.nan_to_num(hmi, nan=0.0)

# 2. Heliocentric angles + correction
theta_deg, cos_theta = compute_heliocentric_angles(aia_map)
hmi_corr, angle_mask, _ = apply_angle_correction(
    hmi, theta_deg, cos_theta, angle_correction=True, mask_over_60=False,
)

# 3. U-Net CH mask (generate pmap on the fly)
smoothing_params = get_postprocessing_params("P1")
pmap = fits_to_pmap(model, aia)
nn_mask = (pmap_to_mask(pmap, smoothing_params) > 0.5) & angle_mask
nn_labeled, nn_n_holes = ndi.label(nn_mask)
nn_holes = extract_hole_data(nn_labeled, nn_n_holes, hmi_corr)

# 4. Plot (no histograms)
plot_multiband(
    aia,
    hmi_corr,
    nn_labeled,
    nn_holes,
    corr_label="B_r (angle-corrected)",
    aia_map=aia_map,
)

print(f"{nn_n_holes} coronal holes detected")

## Orientation audit — visual proof that all arrays are north-up

**The flip contract in this codebase:**

| Stage | Row 0 means | Why |
|---|---|---|
| `sunpy.map.Map(path).data` | **south** (FITS native) | FITS stores row 0 = bottom |
| `prepare_fits()` → `map_data` | **north** (display) | explicit `np.flipud` |
| `reproject_interp(hmi, aia.wcs, …)` | **south** (FITS native) | reproject honours the WCS, not the flipped array |
| notebook `hmi` after `np.flipud` | **north** (display) | explicit flip after reproject |
| `compute_heliocentric_angles()` → θ, cos θ | **north** (display) | explicit `np.flipud` |
| `pmap` / `nn_mask` / `nn_labeled` | **north** (display) | derived from flipped AIA |
| `plot_multiband` → `imshow(…)` | `origin='upper'` (default) | row 0 at top = north at top ✓ |
| Library/Plot.py `plot_ch_map` | mask gets `flipud` → `origin='lower'` | south at bottom ⇒ north at top ✓ |
| `prepare_mask()` (PNG) | **north** row 0 = top of PNG | PNG saved from display orientation |

**Tests below inject a known marker in the top rows and trace it through every stage.**

In [ ]:
# ============================================================
#  TEST 1 — sunpy.map.plot vs raw .data vs prepare_fits
#  Confirms: sunpy handles orientation; prepare_fits flipud is correct
# ============================================================
from astropy.io import fits as astrofits

aia_map_t = sunpy.map.Map(aia_path)
_, aia_t = prepare_fits(aia_path)

fig = plt.figure(figsize=(15, 5))

# Panel A: sunpy's own plot (WCS-aware, always correct)
ax0 = fig.add_subplot(1, 3, 1, projection=aia_map_t)
aia_map_t.plot(axes=ax0, clip_interval=(1, 99.99) * u.percent)
ax0.set_title("A) sunpy .plot()  [reference]")

# Panel B: raw .data with origin='upper' (default) → south at top → WRONG
ax1 = fig.add_subplot(1, 3, 2)
ax1.imshow(aia_map_t.data, cmap=cm.cmlist.get("sdoaia193"), origin="upper")
ax1.set_title("B) raw .data, origin='upper'\n(south at top → WRONG)")

# Panel C: prepare_fits output with origin='upper' → should match panel A
ax2 = fig.add_subplot(1, 3, 3)
ax2.imshow(aia_t, cmap=cm.cmlist.get("sdoaia193"), origin="upper")
ax2.set_title("C) prepare_fits output, origin='upper'\n(flipud applied → CORRECT)")

for ax in [ax1, ax2]:
    ax.set_aspect("equal")
plt.tight_layout()
plt.show()

print("✓ Panel A = reference.  Panel C should match A.  Panel B is intentionally wrong (upside down).")

In [ ]:
# ============================================================
#  TEST 2 — White-rectangle injection
#  Burn a white stripe into the TOP 50 rows of each array,
#  then display everything with the same imshow convention
#  used by plot_multiband (origin='upper').
#  If the stripe is at the TOP in every panel → orientation is consistent.
# ============================================================

def inject_marker(arr, rows=50, value=None):
    """Return a copy with the first `rows` rows set to `value`."""
    out = arr.copy()
    if value is None:
        value = np.nanmax(out)
    out[:rows, :] = value
    return out

# Arrays from the previous standalone cell (already in kernel)
aia_marked    = inject_marker(aia, value=1.0)
hmi_marked    = inject_marker(hmi_corr, value=500)
theta_marked  = inject_marker(theta_deg, value=90)
mask_marked   = inject_marker(nn_mask.astype(float), value=1.0)

hmi_m_norm = np.clip(hmi_marked, -1000, 1000)
hmi_m_norm = (hmi_m_norm + 1000) / 2000

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
titles = [
    "AIA 193 (prepare_fits)",
    "HMI B_r (corrected)",
    "θ heliocentric angle",
    "U-Net CH mask",
]
images = [aia_marked, hmi_m_norm, theta_marked, mask_marked]
cmaps  = [cm.cmlist.get("sdoaia193"), cm.cmlist["hmimag"], "inferno", "gray"]

for ax, img, cmp, ttl in zip(axes, images, cmaps, titles):
    ax.imshow(img, cmap=cmp, origin="upper")       # same convention as plot_multiband
    ax.set_title(ttl)
    ax.axhline(50, color="lime", lw=1, ls="--")    # mark the injected boundary
    ax.text(img.shape[1] // 2, 25, "← injected N stripe",
            color="lime", fontsize=10, ha="center", va="center", weight="bold")
    ax.axis("off")

plt.suptitle("TEST 2 — white stripe injected into rows 0-49  (should be at TOP in all panels)",
             fontsize=13, weight="bold")
plt.tight_layout()
plt.show()

print("✓ If the lime line + bright stripe is at the TOP in every panel, all arrays share the same orientation.")

In [ ]:
# ============================================================
#  TEST 3 — WCS north-pole marker
#  Use the sunpy WCS to compute where heliographic N pole falls
#  in pixel coordinates, then plot a crosshair on every array.
#  This is the definitive test: the N-pole marker must be at TOP.
# ============================================================
from sunpy.coordinates import frames as sf

# Where is the N pole in pixel coords?
north_hgs = SkyCoord(0 * u.deg, 90 * u.deg,
                     frame=sf.HeliographicStonyhurst,
                     obstime=aia_map.date)
north_hpc = north_hgs.transform_to(aia_map.coordinate_frame)
npx = aia_map.world_to_pixel(north_hpc)
nx_pix, ny_pix_fits = float(npx.x.value), float(npx.y.value)

# In FITS coords, y=0 is bottom.  prepare_fits flips → display y = (Ny - 1) - fits_y
Ny = aia.shape[0]
ny_pix_display = (Ny - 1) - ny_pix_fits

print(f"North pole pixel coords:  FITS (x, y) = ({nx_pix:.0f}, {ny_pix_fits:.0f})")
print(f"                          display (x, y) = ({nx_pix:.0f}, {ny_pix_display:.0f})")
print(f"Array height = {Ny}")
print(f"→ display y ~ {ny_pix_display:.0f}  (should be near 0 = top of image)\n")

# Similarly mark the S pole
south_hgs = SkyCoord(0 * u.deg, -90 * u.deg,
                     frame=sf.HeliographicStonyhurst,
                     obstime=aia_map.date)
south_hpc = south_hgs.transform_to(aia_map.coordinate_frame)
spx = aia_map.world_to_pixel(south_hpc)
sx_pix = float(spx.x.value)
sy_pix_display = (Ny - 1) - float(spx.y.value)

hmi_norm_t = np.clip(hmi_corr, -1000, 1000)
hmi_norm_t = (hmi_norm_t + 1000) / 2000

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
panels = [
    ("AIA 193 (prepare_fits)", aia, cm.cmlist.get("sdoaia193")),
    ("HMI B_r (corrected)", hmi_norm_t, cm.cmlist["hmimag"]),
    ("U-Net CH mask", nn_mask.astype(float), "gray"),
]
for ax, (ttl, img, cmp) in zip(axes, panels):
    ax.imshow(img, cmap=cmp, origin="upper")
    # N-pole marker
    ax.plot(nx_pix, ny_pix_display, marker="v", color="cyan", markersize=14, zorder=10)
    ax.annotate("N", (nx_pix, ny_pix_display), textcoords="offset points",
                xytext=(10, -5), color="cyan", fontsize=14, weight="bold")
    # S-pole marker
    ax.plot(sx_pix, sy_pix_display, marker="^", color="magenta", markersize=14, zorder=10)
    ax.annotate("S", (sx_pix, sy_pix_display), textcoords="offset points",
                xytext=(10, 5), color="magenta", fontsize=14, weight="bold")
    ax.set_title(ttl)
    ax.axis("off")

plt.suptitle("TEST 3 — WCS-derived N/S pole markers  (N at top, S at bottom = correct)",
             fontsize=13, weight="bold")
plt.tight_layout()
plt.show()

print("✓ Cyan ▼ = N pole (should be at top).  Magenta ▲ = S pole (should be at bottom).")

In [ ]:
# ============================================================
#  TEST 4 — AIA vs HMI active-region alignment
#  Bright AIA 193 patches should spatially coincide with strong
#  HMI B-field regions.  Also shows AIA 304 if available.
#  This is the physical-consistency check.
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# AIA 193 with HMI contours at ±50 G
axes[0].imshow(aia, cmap=cm.cmlist.get("sdoaia193"), origin="upper")
axes[0].contour(hmi_corr, levels=[-50], colors="blue", linewidths=0.5, alpha=0.6)
axes[0].contour(hmi_corr, levels=[50], colors="red", linewidths=0.5, alpha=0.6)
axes[0].set_title("AIA 193 + HMI ±50G contours\n(red=+B, blue=−B)")
axes[0].axis("off")

# HMI with AIA brightness contours
aia_thresh = np.percentile(aia[aia > 0], 90)
axes[1].imshow(hmi_norm_t, cmap=cm.cmlist["hmimag"], origin="upper")
axes[1].contour(aia, levels=[aia_thresh], colors="orange", linewidths=0.8, alpha=0.7)
axes[1].set_title(f"HMI + AIA 193 bright contour (>{aia_thresh:.2f})\n(active regions should align)")
axes[1].axis("off")

plt.suptitle("TEST 4 — AIA↔HMI cross-alignment  (active regions must coincide)",
             fontsize=13, weight="bold")
plt.tight_layout()
plt.show()

print("✓ If red/blue HMI contours sit on top of AIA bright/active regions, the two arrays are co-aligned.")

In [ ]:
# ============================================================
#  TEST 5 — reproject_interp orientation proof
#  Shows that reproject outputs in FITS convention (south at row 0)
#  and that the subsequent flipud is necessary and correct.
# ============================================================

hmi_map_t = sunpy.map.Map(hmi_path)
hmi_reproj_raw, _ = reproject_interp(hmi_map_t, aia_map.wcs, shape_out=aia.shape)
hmi_reproj_raw = np.nan_to_num(hmi_reproj_raw.astype(np.float32), nan=0.0)

hmi_reproj_flipped = np.flipud(hmi_reproj_raw)

fig = plt.figure(figsize=(15, 5))

# Panel A: sunpy HMI plot (WCS-aware reference)
ax0 = fig.add_subplot(1, 3, 1, projection=hmi_map_t)
hmi_map_t.plot_settings["norm"].vmin = -500
hmi_map_t.plot_settings["norm"].vmax = 500
hmi_map_t.plot(axes=ax0, cmap=cm.cmlist["hmimag"])
ax0.set_title("A) sunpy HMI .plot()  [reference]")

# Panel B: reproject output, origin='upper' → south at top → WRONG
norm = lambda h: (np.clip(h, -500, 500) + 500) / 1000
ax1 = fig.add_subplot(1, 3, 2)
ax1.imshow(norm(hmi_reproj_raw), cmap=cm.cmlist["hmimag"], origin="upper")
ax1.set_title("B) reproject raw, origin='upper'\n(south at top → WRONG)")

# Panel C: after flipud, origin='upper' → north at top → CORRECT
ax2 = fig.add_subplot(1, 3, 3)
ax2.imshow(norm(hmi_reproj_flipped), cmap=cm.cmlist["hmimag"], origin="upper")
ax2.set_title("C) after flipud, origin='upper'\n(north at top → CORRECT)")

for ax in [ax0, ax1, ax2]:
    ax.set_aspect("equal")
plt.suptitle("TEST 5 — reproject_interp needs flipud to get north-up",
             fontsize=13, weight="bold")
plt.tight_layout()
plt.show()

print("✓ Panel A = reference.  Panel C should match A.  Panel B is the raw reproject output (upside down).")

In [ ]:
# ============================================================
#  TEST 6 — Library/Plot.py convention check
#  plot_ch_map does flipud(mask) + origin='lower'.
#  Verify this produces the same visual result as the notebook's
#  origin='upper' convention (no flipud).
# ============================================================

# Simulate what plot_ch_map does: flipud then origin='lower'
mask_for_plotlib = np.flipud(nn_mask.astype(float))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Panel A: notebook convention (what plot_multiband does)
axes[0].imshow(nn_mask.astype(float), cmap="gray", origin="upper")
axes[0].set_title("A) nn_mask, origin='upper'\n(notebook convention)")

# Panel B: Library/Plot.py convention
axes[1].imshow(mask_for_plotlib, cmap="gray", origin="lower")
axes[1].set_title("B) flipud(nn_mask), origin='lower'\n(Library/Plot.py convention)")

# Panel C: WRONG combo — flipud + origin='upper' (would be upside down)
axes[2].imshow(mask_for_plotlib, cmap="gray", origin="upper")
axes[2].set_title("C) flipud(nn_mask), origin='upper'\n(WRONG — upside down)")

for ax in axes:
    ax.set_aspect("equal")
    ax.axis("off")
plt.suptitle("TEST 6 — A and B must match (same visual result, different conventions).  C is intentionally wrong.",
             fontsize=12, weight="bold")
plt.tight_layout()
plt.show()

print("✓ Panels A and B should be identical.  Panel C is intentionally inverted.")